In [1]:
# Importing the libraries
from functions import *

## 1. HMM: Data preparation

In [2]:
# Open XML file as a dataframe
input_file = '{}/databases/uniprot/curated_uniprot.fasta_75'.format(directory) # change the file name if necessary
df = blast_parser(input_file)
print(f"The number of instances: {len(df)}")
df.head(5)

The number of instances: 8656


,query_id,subject_id,query_len,hsp_len,query_seq,match_seq,subject_seq,query_start,query_end,subject_start,subject_end,identity,positive,gaps,eval,bit_score,count
0,Q16620,A0A2R9BM51,822,822,MSSWIRWHGPAMARLWGFCWLVVGFWRAAFACPTSCKCSASRIWCS...,MSSWIRWHGPAMARLWGFCWLVVGFWRAAFACPTSCKCSASRIWCS...,MSSWIRWHGPAMARLWGFCWLVVGFWRAAFACPTSCKCSASRIWCS...,1,822,1,822,822,822,0,0.0,4458.0,200
1,Q16620,A0A4X2LP40,822,824,MSSWIRWHGPAMARLWGFCWLVVGFWRAAFACPTSCKCSASRIWCS...,M SW + HGP MARL GFCWLV+ FWR + ACPTSC CS +RIWCS...,MLSWKKCHGPGMARLLGFCWLVLIFWRGSQACPTSCTCSTTRIWCS...,1,822,1,824,689,745,2,0.0,3749.0,200
2,Q16620,A0A4X2LDU8,822,822,MSSWIRWHGPAMARLWGFCWLVVGFWRAAFACPTSCKCSASRIWCS...,M SW + HGP MARL GFCWLV+ FWR + ACPTSC CS +RIWCS...,MLSWKKCHGPGMARLLGFCWLVLIFWRGSQACPTSCTCSTTRIWCS...,1,822,1,821,689,745,1,0.0,3756.0,200
3,Q16620,A0A6P5IKH1,822,824,MSSWIRWHGPAMARLWGFCWLVVGFWRAAFACPTSCKCSASRIWCS...,M SW + HGP MARL GFCWLV+ FWR + ACPTSC CS +RIWCS...,MLSWKKCHGPGMARLLGFCWLVLIFWRGSQACPTSCTCSTTRIWCS...,1,822,1,824,691,745,2,0.0,3764.0,200
4,Q16620,A0A7J7UQA2,822,741,TSCKCSASRIWCSDPSPGIVAFPRLEPNSVDPENITEIFIANQKRL...,+SCKCSASRIWCSDP PGI+AFPRLEPN++DPENITEI+IANQKRL...,SSCKCSASRIWCSDPIPGIMAFPRLEPNTIDPENITEIYIANQKRL...,34,774,2,739,710,724,3,0.0,3776.0,200


In [3]:
# Create a dataframe with Disprot instances from curated.mjson database
data = list()

with open('curated.mjson', 'r') as file:
    for line in file:
        obj = json.loads(line)
        rows = json_parser(obj)
        data.extend(rows)

curated_disprot = pd.DataFrame(data)

# Calculate the length of disordered regions
curated_disprot['length'] = curated_disprot['end'] - curated_disprot['start'] + 1
curated_disprot = curated_disprot[(curated_disprot['feature'] == 'disorder') & (curated_disprot['source'] == 'disprot')]
curated_disprot.to_csv("curated_disprot.csv", index=False)

print(f"The number of instances in the Curated Disprot database: {len(curated_disprot)}")
curated_disprot.head()

The number of instances in the Curated Disprot database: 3151


,acc,evidence,feature,source,start,end,length
6927,P03265,curated,disorder,disprot,294,334,41
6928,P03265,curated,disorder,disprot,454,464,11
6929,P49913,curated,disorder,disprot,134,170,37
6930,P03045,curated,disorder,disprot,1,107,107
6931,P00004,curated,disorder,disprot,1,105,105


In [4]:
# Keep only disordered regions in the initial dataframe filtering with curated_disordered dataframe
disordered = df[df['query_id'].isin(curated_disprot['acc'])]
disordered.to_csv("disordered_df.csv", index=False)

print(f"The number of rows with the disordered regions: {len(disordered)}")
disordered.head(5)

The number of rows with the disordered regions: 2887


,query_id,subject_id,query_len,hsp_len,query_seq,match_seq,subject_seq,query_start,query_end,subject_start,subject_end,identity,positive,gaps,eval,bit_score,count
200,Q9H832,A0A6J2FM24,354,356,MAESPTEEAATA--GAGAAGPGASSVAGVVGVSGSGGGFGPPFLPD...,MAESPTEEAATA GAGAAGPGAS V GVVGVSGSG FGPPFLPD...,MAESPTEEAATATAGAGAAGPGASGVTGVVGVSGSG--FGPPFLPD...,1,354,1,354,350,350,4,0.0,1851.0,200
201,Q9H832,A0A3Q7W6Y2,354,356,MAESPTEEAATA--GAGAAGPGASSVAGVVGVSGSGGGFGPPFLPD...,MAESPTEEAATA GAGA GPGAS VAGVVGVSGSG FGPPFLPD...,MAESPTEEAATATAGAGATGPGASGVAGVVGVSGSG--FGPPFLPD...,1,354,1,354,350,350,4,0.0,1851.0,200
202,Q9H832,A0A2U3VK69,354,356,MAESPTEEAATA--GAGAAGPGASSVAGVVGVSGSGGGFGPPFLPD...,MAESPTEEAATA GAGAAGPGAS V GVVGVSGSG FGPPFLPD...,MAESPTEEAATATAGAGAAGPGASGVTGVVGVSGSG--FGPPFLPD...,1,354,1,354,350,350,4,0.0,1851.0,200
203,Q9H832,A0A2Y9JVH5,354,358,MAESPTEEAATA----GAGAAGPGASSVAGVVGVSGSGGGFGPPFL...,MAESPTEEAATA GAGAAGPGAS VAGVVGVSGSG FGPPFL...,MAESPTEEAATATATAGAGAAGPGASGVAGVVGVSGSG--FGPPFL...,1,354,1,356,351,351,6,0.0,1854.0,200
204,Q9H832,A0A8C7ALE4,354,358,MAESPTEEAATA----GAGAAGPGASSVAGVVGVSGSGGGFGPPFL...,MAESPTEEAATA GAGAAGPGAS VAGVVGVSGSG FGPPFL...,MAESPTEEAATATATAGAGAAGPGASGVAGVVGVSGSG--FGPPFL...,1,354,1,356,351,351,6,0.0,1854.0,200


In [5]:
# Define an array of disordered regions ids
disprot_ids = disordered["query_id"].unique()

# Dropdown list of Uniprot query IDs for disordered regions
output = widgets.Select(options=disprot_ids,
    rows=10,
    description='Uniprot ID: ',
    layout={'width': 'max-content'},
    disabled=False)

display(output)

Select(description='Uniprot ID: ', layout=Layout(width='max-content'), options=('Q9H832', 'Q8IW19', 'Q99967', …

In [6]:
# Check if there are disordered regions for a particular query ID
id_dis = output.value
i = 1 # change to the necessary region
# store the data
%store id_dis i
selected_dis = disordered[disordered["query_id"].isin([id_dis])]
selected_dis.head()

Stored 'id_dis' (str)
Stored 'i' (int)


,query_id,subject_id,query_len,hsp_len,query_seq,match_seq,subject_seq,query_start,query_end,subject_start,subject_end,identity,positive,gaps,eval,bit_score,count
200,Q9H832,A0A6J2FM24,354,356,MAESPTEEAATA--GAGAAGPGASSVAGVVGVSGSGGGFGPPFLPD...,MAESPTEEAATA GAGAAGPGAS V GVVGVSGSG FGPPFLPD...,MAESPTEEAATATAGAGAAGPGASGVTGVVGVSGSG--FGPPFLPD...,1,354,1,354,350,350,4,0.0,1851.0,200
201,Q9H832,A0A3Q7W6Y2,354,356,MAESPTEEAATA--GAGAAGPGASSVAGVVGVSGSGGGFGPPFLPD...,MAESPTEEAATA GAGA GPGAS VAGVVGVSGSG FGPPFLPD...,MAESPTEEAATATAGAGATGPGASGVAGVVGVSGSG--FGPPFLPD...,1,354,1,354,350,350,4,0.0,1851.0,200
202,Q9H832,A0A2U3VK69,354,356,MAESPTEEAATA--GAGAAGPGASSVAGVVGVSGSGGGFGPPFLPD...,MAESPTEEAATA GAGAAGPGAS V GVVGVSGSG FGPPFLPD...,MAESPTEEAATATAGAGAAGPGASGVTGVVGVSGSG--FGPPFLPD...,1,354,1,354,350,350,4,0.0,1851.0,200
203,Q9H832,A0A2Y9JVH5,354,358,MAESPTEEAATA----GAGAAGPGASSVAGVVGVSGSGGGFGPPFL...,MAESPTEEAATA GAGAAGPGAS VAGVVGVSGSG FGPPFL...,MAESPTEEAATATATAGAGAAGPGASGVAGVVGVSGSG--FGPPFL...,1,354,1,356,351,351,6,0.0,1854.0,200
204,Q9H832,A0A8C7ALE4,354,358,MAESPTEEAATA----GAGAAGPGASSVAGVVGVSGSGGGFGPPFL...,MAESPTEEAATA GAGAAGPGAS VAGVVGVSGSG FGPPFL...,MAESPTEEAATATATAGAGAAGPGASGVAGVVGVSGSG--FGPPFL...,1,354,1,356,351,351,6,0.0,1854.0,200


In [7]:
# Filter the curated_disprot dataframe based on selected ID
curated_query = curated_disprot[curated_disprot['acc'] == id_dis]
curated_query

,acc,evidence,feature,source,start,end,length
7799,Q9H832,curated,disorder,disprot,1,99,99
7800,Q9H832,curated,disorder,disprot,327,354,28


## 2. hmmbuild
We build an HMM of each disordered region, using trimmed MSA as an input.

In [8]:
# Set the paths to HMM and MSA files
hmm_file = f'{directory}/results/hmms/hmmbuild/{id_dis}.hmm'
align_file = f'{directory}/results/alignments/output_files/disordered/{id_dis}_{i}.fasta'

In [9]:
# for id_dis in disprot_ids:
#     hmm_file = f'{directory}/results/hmms/hmmbuild/{id_dis}.hmm'
#     align_file = f'{directory}/results/alignments/output_files/disordered/{id_dis}_{i}.fasta'
#     print(hmm_file, align_file)

In [10]:
# Build HMM
!hmmbuild {hmm_file} {align_file}

# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# input alignment file:             /Users/alina/HMM/results/alignments/output_files/disordered/Q9H832_1.fasta
# output HMM file:                  /Users/alina/HMM/results/hmms/hmmbuild/Q9H832.hmm
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# idx name                  nseq  alen  mlen eff_nseq re/pos description
#---- -------------------- ----- ----- ----- -------- ------ -----------
1     Q9H832_1               201    99    99     2.96  0.590 

# CPU time: 0.04u 0.00s 00:00:00.04 Elapsed: 00:00:00.04


## 3. hmmsearch

After building the model, our objective is to assess if overlaps with the profiles in Reference Proteome 15% exist and to enrich the model by utilizing this database. We generate dataframes containing the most significant sequences, with a default E-value threshold of 0.01.

In [11]:
# # Copy the HMM file to remote computer to perform HMM search on a cluster
# !scp {directory}/results/hmms/hmmbuild/{id_dis}_{i}.hmm {name}@{server}:~/{id_dis}_{i}.hmm

In [12]:
# # hmmsearch against Reference Proteome 15%
# !ssh {name}@{server} "/software/packages/hmmer/hmmer-3.3.2/usr/bin/hmmsearch {id_dis}_{i}.hmm /db/rp/rp-seqs-15.fasta.gz > hmmsearch_rp_15_{id_dis}_{i}.txt"

In [13]:
# # Copy results to the local folder
# !scp {name}@{server}:~/hmmsearch_rp_15_{id_dis}_{i}.txt {directory}/results/hmms/hmmsearch/

In [14]:
# # Remove the files from the remote folder
# !ssh {name}@{server} "rm ~/{id_dis}_{i}.hmm"
# !ssh {name}@{server} "rm ~/hmmsearch_rp_15_{id_dis}_{i}.txt"

### 3.1 hmmsearch results preprocessing (for one alignment)

In [15]:
# 1. Make a dataframe with the statistics - the first table of the hmmsearch output
stats_rp_15 = process_hmmsearch_file(f"{directory}/results/hmms/hmmsearch/hmmsearch_rp_15_{id_dis}_{i}.txt")
# stats_rp_15.to_csv(f"{directory}/results/hmms/hmmsearch/stats/stats_rp_15_{id_dis}_{i}.csv", index=False)
stats_rp_15[:10]

,E-value,score,bias,E-value,score,bias,exp,N,Sequence,Description
0,1.6e-47,169.6,18.6,2.6e-47,168.9,18.6,1.4,1,Q9H832,UBE2Z_HUMAN^|^^|^Ubiquitin-conjugating enzyme E2
1,3.9e-47,168.3,18.6,6.4e-47,167.6,18.6,1.4,1,A0A7N9CX17,A0A7N9CX17_MACFA^|^^|^Ubiquitin conjugating enzym
2,2.5e-45,162.5,18.8,4.1e-45,161.8,18.8,1.4,1,Q3UE37,UBE2Z_MOUSE^|^^|^Ubiquitin-conjugating enzyme E2
3,2.6e-45,162.4,19.1,4.3e-45,161.7,19.1,1.4,1,Q3B7D1,UBE2Z_RAT^|^^|^Ubiquitin-conjugating enzyme E2 Z^
4,5.1e-34,126.2,22.4,9.4e-33,122.2,22.4,2.3,1,A0A2K5VU74,A0A2K5VU74_MACFA^|^^|^Ubiquitin conjugating enzym
5,8.9e-16,67.7,24.2,1.5e-15,67.0,24.2,1.4,1,E1BQ87,E1BQ87_CHICK^|^^|^UBC core domain-containing prot
6,0.0014,28.6,1.1,0.0014,28.6,1.1,2.1,2,A0A2G9R8D0,A0A2G9R8D0_LITCT^|^^|^UBC core domain-containing
7,0.0016,28.4,12.7,0.0016,28.4,12.7,2.3,1,A0A4Z2GIF1,A0A4Z2GIF1_9TELE^|^^|^Ubiquitin-conjugating enzym
8,0.0017,28.3,14.1,1.4e+03,9.4,2.8,6.2,5,A0A0D2AXJ1,A0A0D2AXJ1_9PEZI^|^^|^Uncharacterized protein {EC
9,0.0029,27.6,1.2,0.0029,27.6,1.2,2.3,2,A0A2G9R9W7,A0A2G9R9W7_LITCT^|^^|^Uncharacterized protein {EC


In [16]:
# 2. Create a dataframe with the extracted regions from HMM - from the rest of the file
hmm_result_file = f"{directory}/results/hmms/hmmsearch/hmmsearch_rp_15_{id_dis}_{i}.txt"
hmmsearch_regions = extract_table_from_output(hmm_result_file)
# hmmsearch_regions.to_csv(f"{directory}/results/hmms/hmmsearch/stats/stats_rp_15_regions_{id_dis}_{i}.csv")
# hmmsearch_regions[:10]

In [17]:
# 3. Combine the results into a dataframe and save the file
hmmsearch_results = pd.merge(stats_rp_15, hmmsearch_regions, left_on="Sequence", right_on="id", how="inner")
hmmsearch_results = hmmsearch_results.drop(columns=["Description", "id"])
hmmsearch_results.to_csv(f"{directory}/results/hmms/hmmsearch/stats/stats_rp_15_{id_dis}_{i}.csv", index=False)
print(f"Number of hmmsearch hits for the {i} disordered region of the {id_dis} protein:", len(hmmsearch_results))
%store hmmsearch_results
hmmsearch_results[:10]

Number of hmmsearch hits for the 1 disordered region of the Q9H832 protein: 11
Stored 'hmmsearch_results' (DataFrame)


,E-value,score,bias,E-value,score,bias,exp,N,Sequence,hmm_from,hmm_to,hmm_length
0,1.6e-47,169.6,18.6,2.6e-47,168.9,18.6,1.4,1,Q9H832,1,99,99
1,3.9e-47,168.3,18.6,6.4e-47,167.6,18.6,1.4,1,A0A7N9CX17,1,99,99
2,2.5e-45,162.5,18.8,4.1e-45,161.8,18.8,1.4,1,Q3UE37,1,99,99
3,2.6e-45,162.4,19.1,4.3e-45,161.7,19.1,1.4,1,Q3B7D1,1,99,99
4,5.1e-34,126.2,22.4,9.4e-33,122.2,22.4,2.3,1,A0A2K5VU74,1,99,99
5,8.9e-16,67.7,24.2,1.5e-15,67.0,24.2,1.4,1,E1BQ87,1,98,98
6,0.0014,28.6,1.1,0.0014,28.6,1.1,2.1,2,A0A2G9R8D0,59,99,41
7,0.0016,28.4,12.7,0.0016,28.4,12.7,2.3,1,A0A4Z2GIF1,2,99,98
8,0.0017,28.3,14.1,1.4e+03,9.4,2.8,6.2,5,A0A0D2AXJ1,19,97,79
9,0.0029,27.6,1.2,0.0029,27.6,1.2,2.3,2,A0A2G9R9W7,59,99,41


### 3.2 hmmsearch results preprocessing (overall)

In [18]:
# Combine hmmsearch results for all ids in disprot_ids array
hmmsearch_path = f'{directory}/results/hmms/hmmsearch/stats' # set the path to the files with hmmsearch statistics
dataframes = []

for filename in os.listdir(hmmsearch_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(hmmsearch_path, filename)
        df = pd.read_csv(file_path, sep=',')
        dataframes.append(df)

# Merge the files
hmmsearch_combined = pd.concat(dataframes, ignore_index=True)
hmmsearch_combined = hmmsearch_combined.iloc[:, 1:]
hmmsearch_combined = hmmsearch_combined.dropna(axis=0)
hmmsearch_combined.to_csv('results/hmms/hmmsearch/hmmsearch_combined.csv', index=False)
hmmsearch_combined

,score,bias,E-value.1,score.1,bias.1,exp,N,Sequence,hmm_from,hmm_to,hmm_length
0,169.6,18.6,0.0,168.9,18.6,1.4,1,Q9H832,1,99,99
1,168.3,18.6,0.0,167.6,18.6,1.4,1,A0A7N9CX17,1,99,99
2,162.5,18.8,0.0,161.8,18.8,1.4,1,Q3UE37,1,99,99
3,162.4,19.1,0.0,161.7,19.1,1.4,1,Q3B7D1,1,99,99
4,126.2,22.4,0.0,122.2,22.4,2.3,1,A0A2K5VU74,1,99,99
...,...,...,...,...,...,...,...,...,...,...,...
1340,24.8,0.7,160.0,11.0,0.0,3.1,3,A0A4P9XX39,1,31,31
1341,24.8,0.1,0.019,23.6,0.1,1.7,1,Q8R5I7,2,32,31
1342,24.6,4.7,0.45,19.2,0.2,3.3,3,A0A0G2E5J8,1,32,32
1343,24.6,3.2,0.0093,24.6,3.2,2.6,1,A0A0D2W1I1,1,32,32


In [19]:
# # Combine all hmmsearch results
# # hmmsearch_path = f'{directory}/results/hmms/hmmsearch/stats'
# dataframes = []

# for filename in os.listdir(hmmsearch_path):
#     if filename.startswith('stats_rp_15_regions') and filename.endswith('.csv'):
#         file_path = os.path.join(hmmsearch_path, filename)
#         df = pd.read_csv(file_path)
#         dataframes.append(df)
        
# combined_data = pd.concat(dataframes, ignore_index=True)
# combined_data = combined_data.iloc[:, 1:]
# duplicates = combined_data[combined_data[['id', 'hmm_length']].duplicated(keep='first')]
# combined_data.drop(duplicates.index, inplace=True)
# combined_data.to_csv('results/hmms/hmmsearch/combined_data.csv', index=False)
# combined_data

In [20]:
# # Extract the list of unique Uniprot Ids
# unique_ids = combined_data['id'].unique()
# unique_ids_df = pd.DataFrame({'id': unique_ids})
# unique_ids_df.to_csv('unique_ids.csv', index=False)

These Uniprot IDs are used in extracting the Pfam regions from Interpro.

## 4. Pfam
### 4.1 Pfam results preprocessing (for one alignment)

In [21]:
# # Copy again the new file to the remote computer
# !scp {directory}/results/hmms/hmmsearch/stats/stats_rp_15_{id_dis}_{i}.csv {name}@{server}:~/stats_rp_15_{id_dis}_{i}.csv

In [22]:
# # Check the overlap of the retrieved regions in RP with protein2ipr database
# !ssh {name}@{server} "/home/alina/protein2ipr_iterator.py stats_rp_15_{id_dis}_{i}.csv /db/interpro/protein2ipr.dat.gz protein2ipr_rp_15_{id_dis}_{i}.txt"

In [23]:
# # Copy the files with overlapping regions to the local folder
# !scp {name}@{server}:~/protein2ipr_rp_15_{id_dis}_{i}.txt {directory}/results/hmms/hmmsearch/protein2ipr

In [24]:
# # Remove the files from the remote folder
# !ssh {name}@{server} "rm ~/stats_rp_15_{id_dis}_{i}.csv"

In [25]:
# Filter only entries with Pfam ID and intercepting regions with the curated_disprot instances
filename = f"{directory}/results/hmms/hmmsearch/protein2ipr/protein2ipr_rp_15_{id_dis}_{i}.txt"
pfam = read_and_filter_pfam_data(filename, curated_query) # adding filtering by "PF" in "pfam_id" column
if len(pfam) != 0: # if there exists an overlap of hmmsearch results and pfam
    pfam.to_csv(f"results/pfam/pfam_overlap/pfam_overlap_{id_dis}.csv", index=False)
print(f"The number of overlapping entries with Pfam instances for the {id_dis} protein: {len(pfam)}")
pfam

The number of overlapping entries with Pfam instances for the Q9H832 protein: 12


,uniprot_id,ipr_id,description,pfam_id,start_pos,end_pos,length
0,Q9H832,IPR000608,Ubiquitin-conjugating enzyme E2,PF00179,103,231,129
1,A0A444UQ25,IPR000608,Ubiquitin-conjugating enzyme E2,PF00179,178,306,129
2,A0A4W3JHM2,IPR000608,Ubiquitin-conjugating enzyme E2,PF00179,92,220,129
3,A0A4Z2GIF1,IPR000608,Ubiquitin-conjugating enzyme E2,PF00179,133,264,132
4,A0A7N9CX17,IPR000608,Ubiquitin-conjugating enzyme E2,PF00179,103,231,129
5,A0A812B0C6,IPR000608,Ubiquitin-conjugating enzyme E2,PF00179,49,176,128
6,A0A8J4TYB0,IPR000608,Ubiquitin-conjugating enzyme E2,PF00179,125,253,129
7,H3B9H2,IPR000608,Ubiquitin-conjugating enzyme E2,PF00179,128,256,129
8,Q3B7D1,IPR000608,Ubiquitin-conjugating enzyme E2,PF00179,105,233,129
9,Q3UE37,IPR000608,Ubiquitin-conjugating enzyme E2,PF00179,105,233,129


### 4.2 Pfam results preprocessing (overall)

In [26]:
# Combine all pfam results
pfam_path = f'{directory}/results/pfam/pfam_overlap'
dataframes = []

for filename in os.listdir(pfam_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(pfam_path, filename)
        df = pd.read_csv(file_path)
        dataframes.append(df)
        
pfam_combined = pd.concat(dataframes, ignore_index=True)
pfam_combined.to_csv('results/pfam/pfam_combined.csv', index=False)
pfam_combined

,uniprot_id,ipr_id,description,pfam_id,start_pos,end_pos,length
0,A0A2G8JLS5,IPR019406,"Aprataxin and PNK-like factor, PBZ domain",PF10283,415,440,26
1,A0A2G8JLS5,IPR019406,"Aprataxin and PNK-like factor, PBZ domain",PF10283,458,480,23
2,A0A2G8JLS5,IPR041388,"PNK, FHA domain",PF17913,19,93,75
3,A0A2K5VR45,IPR019406,"Aprataxin and PNK-like factor, PBZ domain",PF10283,375,400,26
4,A0A2K5VR45,IPR019406,"Aprataxin and PNK-like factor, PBZ domain",PF10283,418,439,22
...,...,...,...,...,...,...,...
3884,V4A7B4,IPR003892,Ubiquitin system component CUE,PF02845,236,276,41
3885,V8NJ04,IPR000008,C2 domain,PF00168,86,187,102
3886,V8NJ04,IPR003892,Ubiquitin system component CUE,PF02845,263,302,40
3887,W4Z0N8,IPR000008,C2 domain,PF00168,65,166,102


In [27]:
# Dataframe of HMM and Pfam instances
pfam_hmm = pd.merge(pfam_combined, hmmsearch_combined[['Sequence', 'hmm_from', 'hmm_to', 'hmm_length']], left_on='uniprot_id', right_on='Sequence', how='left')
pfam_hmm = pfam_hmm.dropna(axis=0)
pfam_hmm = pfam_hmm.drop(columns='uniprot_id')
pfam_hmm.to_csv('results/pfam/pfam_hmm.csv', index=False)
print(len(pfam_hmm))
pfam_hmm[:20]

2374


,ipr_id,description,pfam_id,start_pos,end_pos,length,Sequence,hmm_from,hmm_to,hmm_length
0,IPR019406,"Aprataxin and PNK-like factor, PBZ domain",PF10283,415,440,26,A0A2G8JLS5,4,62,59
1,IPR019406,"Aprataxin and PNK-like factor, PBZ domain",PF10283,458,480,23,A0A2G8JLS5,4,62,59
2,IPR041388,"PNK, FHA domain",PF17913,19,93,75,A0A2G8JLS5,4,62,59
3,IPR019406,"Aprataxin and PNK-like factor, PBZ domain",PF10283,375,400,26,A0A2K5VR45,1,62,62
4,IPR019406,"Aprataxin and PNK-like factor, PBZ domain",PF10283,375,400,26,A0A2K5VR45,1,22,22
5,IPR019406,"Aprataxin and PNK-like factor, PBZ domain",PF10283,418,439,22,A0A2K5VR45,1,62,62
6,IPR019406,"Aprataxin and PNK-like factor, PBZ domain",PF10283,418,439,22,A0A2K5VR45,1,22,22
7,IPR041388,"PNK, FHA domain",PF17913,20,99,80,A0A2K5VR45,1,62,62
8,IPR041388,"PNK, FHA domain",PF17913,20,99,80,A0A2K5VR45,1,22,22
9,IPR019406,"Aprataxin and PNK-like factor, PBZ domain",PF10283,365,390,26,A0A2K6T215,1,62,62


In [29]:
pfam_hmm[pfam_hmm['Sequence'] == 'Q9H832']

,ipr_id,description,pfam_id,start_pos,end_pos,length,Sequence,hmm_from,hmm_to,hmm_length
97,IPR000608,Ubiquitin-conjugating enzyme E2,PF00179,103,231,129,Q9H832,1,99,99
98,IPR000608,Ubiquitin-conjugating enzyme E2,PF00179,103,231,129,Q9H832,1,28,28
112,IPR000608,Ubiquitin-conjugating enzyme E2,PF00179,103,231,129,Q9H832,1,99,99
113,IPR000608,Ubiquitin-conjugating enzyme E2,PF00179,103,231,129,Q9H832,1,28,28
